---
title: IDs from STEREO
format:
  html:
    code-fold: true
output-file: stereo.html
---

Need to run command in shell first as `pipeline` is project-specific command

```{sh}
kedro pipeline create stearo
```

In [3]:
#| default_exp pipelines/stearo/pipeline

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy

In [ ]:
#| code-summary: import all the packages needed for the project
#| export
#| output: hide
import os

from fastcore.utils import *
from fastcore.test import *
from ids_finder.utils.basic import partition_data_by_year
from ids_finder.core import *

import polars as pl
import pandas
import numpy as np

from datetime import timedelta
from loguru import logger

from typing import Callable

os.environ['SPEDAS_DATA_DIR'] = f"{os.environ['HOME']}/data"

#### `Kerdo`

In [ ]:
#| export
from kedro.pipeline import Pipeline, pipeline, node
from ids_finder.pipelines.data_processing.pipeline import extract_features

## Dataset Overview

In [ ]:
jno_start_date = catalog.load('params:jno_start_date')
jno_end_date = catalog.load('params:jno_end_date')
trange = [jno_start_date, jno_end_date]
test_trange = ["2011-08-25", "2011-09-01"]

### Download and transform magnetic field and state data

Download data using `pyspedas`, but load it using `pycdfpp` (using `pyspedas` to load the data directly into `xarray` is very slow)

When downsampling, the magnetic field data is too large to process at once, split it into multiple time intervals.

Using `wget` does not work.

`wget --recursive --no-parent --no-clobber http://sprg.ssl.berkeley.edu/data/misc/stereo/impact/level1/ahead/mag/RTN/2014`

In [ ]:

#| export
import pyspedas
import pycdfpp
from sunpy.time import TimeRange
from pipe import select

In [ ]:
# | export
def spedas2pl(trange, pyspedas_func: Callable, probe="a"):
    if isinstance(trange, TimeRange):
        trange = [trange.start.strftime("%Y-%m-%d"), trange.end.strftime("%Y-%m-%d")]

    files = pyspedas_func(trange, downloadonly=True, probe=probe)

    cdfs = [pycdfpp.load(file) for file in files]
    times = [pycdfpp.to_datetime64(cdf["Epoch"]) for cdf in cdfs]
    BFIELDs = [cdf["BFIELD"].values for cdf in cdfs]

    time = np.concatenate(times)
    BFIELD = np.concatenate(BFIELDs)

    df = pl.DataFrame(
        {
            "time": time,
            "BX": BFIELD[:, 0],
            "BY": BFIELD[:, 1],
            "BZ": BFIELD[:, 2],
            "B": BFIELD[:, 3],
        }
    )
    return df


def get_stereo_mag_data(
    tstart, tend, every: int = 1  # in units of seconds
) -> pl.DataFrame:
    """Get stereo downsampled magnetic field data"""

    trange = [tstart, tend]
    every = timedelta(seconds=every)
    period = 2 * every

    df_downsampled = pl.concat(
        TimeRange(trange).split(16)
        | select(partial(spedas2pl, pyspedas_func=pyspedas.stereo.mag))
        | select(partial(resample, every=every, period=period))
    )
      
    return df_downsampled

In [ ]:
# | export
def download_stereo_state_data(tstart, tend):
    import pooch

    download = partial(pooch.retrieve, known_hash=None)

    start_time = pandas.Timestamp(tstart)
    end_time = pandas.Timestamp(tend)

    url = "https://spdf.gsfc.nasa.gov/pub/data/stereo/ahead/l2/merged/stereoa{year}.asc"

    files = list(
        range(start_time.year, end_time.year + 1)
        | select(lambda x: url.format(year=x))
        | select(download)
    )
    return files


parameters = """Year
DOY
Hour
Radial Distance, AU
HGI Lat. of the S/C
HGI Long. of the S/C
IMF BR, nT (RTN)
IMF BT, nT (RTN)
IMF BN, nT (RTN)
IMF B Scalar, nT
SW Plasma Speed, km/s
SW Lat. Angle RTN, deg.
SW Long. Angle RTN, deg.
SW Plasma Density, N/cm^3
SW Plasma Temperature, K
1.8-3.6 MeV H flux,LET
4.0-6.0 MeV H flux,LET
6.0-10.0 MeV H flux, LET
10.0-12.0 MeV H flux,LET
13.6-15.1 MeV H flux, HET
14.9-17.1 MeV H flux, HET
17.0-19.3 MeV H flux, HET
20.8-23.8 MeV H flux, HET
23.8-26.4 MeV H flux, HET
26.3-29.7 MeV H flux, HET
29.5-33.4 MeV H flux, HET
33.4-35.8 MeV H flux, HET
35.5-40.5 MeV H flux, HET
40.0-60.0 MeV H flux, HET
60.0-100.0 MeV H flux, HET
0.320-0.452 MeV H flux, SIT
0.452-0.64 MeV H flux, SIT
0.640-0.905 MeV H flux, SIT
0.905-1.28 MeV H flux, SIT
1.280-1.81 MeV H flux, SIT
1.810-2.56 MeV H flux, SIT
2.560-3.62 MeV H flux, SIT"""


def get_stereo_state_data(tstart, tend) -> pl.DataFrame:
    files = download_stereo_state_data(tstart, tend)
    columns = parameters.split("\n")

    df = pl.concat(
        files
        | select(partial(pandas.read_csv, delim_whitespace=True, names=columns))
        | select(pl.from_pandas)
    )
    
    return df.select(
        ["Radial Distance, AU", "HGI Lat. of the S/C", "HGI Long. of the S/C"],
        time=(
            pl.datetime(pl.col("Year"), month=1, day=1)
            + pl.duration(days=pl.col("DOY") - 1, hours=pl.col("Hour"))
        ).dt.cast_time_unit("ns"),
    )

In [ ]:
# download_stereo_state_data(*trange)
# get_stereo_state_data(*trange)

## Processing the whole data

### Combine STEREO magnetic field and state data

In [ ]:
# | export
def combine_stereo_data(mag_df: pl.DataFrame, state_df) -> dict:
    result = (
        mag_df.lazy()
        .with_columns(pl.col("time").dt.cast_time_unit("ns"))
        .sort("time")
        .join_asof(state_df.lazy().sort("time"), on="time")
    )

    # TODO: having some overlap between partitions would be nice
    # trange = [f"{year}-01-01", f"{year+1}-01-01T01"]
    
    return partition_data_by_year(result)

### Nodes

In [ ]:
# | export
node_sta_get_mag_data = node(
    get_stereo_mag_data,
    inputs=["params:jno_start_date", "params:jno_end_date"],
    outputs="sta_mag_rtn_1s",
    name="get_STEREO_mag_data",
)

node_sta_get_state_data = node(
    get_stereo_state_data,
    inputs=["params:jno_start_date", "params:jno_end_date"],
    outputs="sta_state_rtn_1h",
    name="get_STEREO_state_data",
)

node_sta_combine_mag_state_data = node(
    combine_stereo_data,
    inputs=["sta_mag_rtn_1s", "sta_state_rtn_1h"],
    outputs="primary_sta_rtn_1s",
    name="commbine_STEREO_mag_and_state_data",
)

node_sta_extract_features = node(
    extract_features,
    inputs=["primary_sta_rtn_1s", "params:tau", "params:sta_1s_params"],
    outputs="candidates_sta_rtn_1s",
    name="extract_STEREO-A_features",
)

In [ ]:
# | export
def create_pipeline(**kwargs) -> Pipeline:
    sta_pipeline = pipeline(
        [
            node_sta_get_mag_data,
            node_sta_get_state_data,
            node_sta_combine_mag_state_data,
            node_sta_extract_features,
        ]
    )

    return sta_pipeline

In [ ]:
# kedro run --to-outputs=sta_mag_rtn_1s
# kedro run --to-outputs=sta_state_rtn_1s
# kedro run --from-inputs=sta_mag_rtn_1s --to-outputs=primary_sta_rtn_1s

# sta_state_rtn_1h = catalog.load('sta_state_rtn_1h')
# sta_mag_rtn_1s = catalog.load('sta_mag_rtn_1s')

# get_memory_usage(sta_mag_rtn_1s)

## Caveats

> Note that for missing data, fill values consisting of a blank followed 
by 9's which together constitute the format are used

## Obsolete codes

NOTE: one can also use `speasy` to download data, however this is slower for `STEREO` data.

In [ ]:
%%markdown
sat_fgm_product = cda_tree.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.BFIELD
sat_fgm_product = 'cda/STA_L1_MAG_RTN/BFIELD'
products = [sat_fgm_product]

dataset = spz.get_data(products, test_trange, disable_proxy=True)
sat_fgm_data  = dataset[0]
data_preview(sat_fgm_data)

Download data in a background thread

In [ ]:
%%markdown

@threaded
def download_data(products, trange):
    logger.info("Downloading data")
    spz.get_data(products, trange, disable_proxy=True)
    logger.info("Data downloaded")
    
download_data(products, trange)

In [ ]:
import speasy as spz

In [ ]:
cda_tree: spz.SpeasyIndex = spz.inventories.tree.cda
product = cda_tree.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN

logger.info(product.description)
logger.info(product.ID)
logger.info(product.BFIELD.CATDESC)
logger.info(product.BFIELD.spz_uid())

# spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.
# spz.inventories.data_tree.cda.STEREO.STEREOA.IMPACT_MAG.STA_LB_MAG_RTN.description
# spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.MAGFLAGUC.CATDESC
spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.BFIELD.CATDESC
# spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.BFIELD.